In [1]:
#Импорт библиотек
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

# Context/Контекст

Проанализируем синтетический набор данных, основанный на реальном наборе данных о пассажирах, которые плыли на корабле Titanic.

In [41]:
#Загрузка данных 
train_data = pd.read_csv(r'D:\RSSchool\2_data_exploration\train.csv')
train_data.shape
train_data.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,1,"Oconnor, Frankie",male,NaN,2,0,209245,27.14,C12239,S
1,1,0,3,"Bryan, Drew",male,NaN,0,0,27323,13.35,NaN,S
2,2,0,3,"Owens, Kenneth",male,0.33,1,2,CA 457703,71.29,NaN,S
3,3,0,3,"Kramer, James",male,19.00,0,0,A. 10866,13.04,NaN,S
4,4,1,3,"Bond, Michael",male,25.00,0,0,427635,7.76,NaN,S


## Описание переменных:

__PassengerId__ — уникальный ID пассажира;

__Survived__ — выжил пассажир или нет (0 - нет, 1 - да);

__Pclass__ — класс билета (1 - 1-ый класс, 2 - 2-ой класс; 3 - 3-ий класс);

__Name__ — имя пассажира;

__Sex__ — пол пассажира (male - мужской, female - женский);

__Age__ — возраст пассажира;

__SibSp__ — количество братьев и сестер / супругов на борту Титаника;

__Parch__ — количество родителей / детей на борту Титаника;

__Ticket__ — номер билета;

__Fare__ — стоимость билета;

__Cabin__ — номер каюты;

__Embarked__ — порт посадки (C - Шербур, Q = Квинстаун, S = Саутгемптон) . 

Всего 11 переменных (не считая __PassengerId__).
Из них 7 категориальных переменных. 4 числовые переменные. 
Всего наблюдений 100000.


# Data quality assessment/Оценка качества данных

In [8]:
#Общая классификация данных, информация о типах данных
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   PassengerId  100000 non-null  int64  
 1   Survived     100000 non-null  int64  
 2   Pclass       100000 non-null  int64  
 3   Name         100000 non-null  object 
 4   Sex          100000 non-null  object 
 5   Age          96708 non-null   float64
 6   SibSp        100000 non-null  int64  
 7   Parch        100000 non-null  int64  
 8   Ticket       95377 non-null   object 
 9   Fare         99866 non-null   float64
 10  Cabin        32134 non-null   object 
 11  Embarked     99750 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 9.2+ MB


Исходя из представленных данных можем сделать вывод о том, что столбцы Age, Ticket, Fare, Cabin, Embarked заполнены неполностью, имеются потерянные данные. Больше всего данных потеряно в стобце Cabin, поэтому рационально будет удалить этот стобец. Столбец Age имеет тип float64.

In [33]:
#Поиск дубликатов в данных
train_data[train_data.duplicated()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


Дубликатов не выявлено

In [34]:
#Уникальные значения столбца
train_data.Name.value_counts()


Johnson, John        31
Smith, James         24
Smith, John          24
Johnson, William     22
Smith, Michael       20
                     ..
Davis, Andre          1
Rosenblum, Ronald     1
Landry, Ernesto       1
Mcdonald, Frances     1
Enciso, Tyler         1
Name: Name, Length: 92144, dtype: int64

Переменные Ticket, Name, PassengerId уникальные для каждого пассажира и не несут какой-либо статистической информации. Удалим эти стобцы и перепишем все в новый датафрейм. 

In [36]:
#Копирование датафрейма и удаление из него неинформативных столбцов
synthanic_data = train_data.copy()
synthanic_data.drop (['PassengerId', 'Name', 'Ticket', 'Cabin'], axis = 1, inplace = True)
synthanic_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,1,male,NaN,2,0,27.14,S
1,0,3,male,NaN,0,0,13.35,S
2,0,3,male,0.33,1,2,71.29,S
3,0,3,male,19.00,0,0,13.04,S
4,1,3,male,25.00,0,0,7.76,S


In [38]:
#Описательная статистика данных
synthanic_data.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,100000.000000,100000.000000,96708.000000,100000.000000,100000.000000,99866.00000
mean,0.427740,2.106910,38.355472,0.397690,0.454560,43.92933
std,0.494753,0.837727,18.313556,0.862566,0.950076,69.58882
min,0.000000,1.000000,0.080000,0.000000,0.000000,0.68000
25%,0.000000,1.000000,25.000000,0.000000,0.000000,10.04000
50%,0.000000,2.000000,39.000000,0.000000,0.000000,24.46000
75%,1.000000,3.000000,53.000000,1.000000,1.000000,33.50000
max,1.000000,3.000000,87.000000,8.000000,9.000000,744.66000


Исходя из описательной статистики можно сделать вывод о том, что пассажиры на борту разновозрастные от младеца до 87летнего человека.

Максимальное значение SibSp	= 8, это значит, что на борту были семьи. 

Цена билета варьируется от 0.68 до 744.66, но среднее mean=43.92933. Это значит, что большинство билетов стоило много меньше самого дорогого. 

# Data exploration/Исследование данных

# Summary/Итоги